# IBM Capstone Project1

In [1]:
##import packages
import pandas as pd
import numpy as np
import requests

### Install wikipedia package to load data 
### Then reserve it in dataframe

In [8]:
!pip install wikipedia
import wikipedia as wp
 
#Get the html source
html = wp.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")
df = pd.read_html(html)[0]

print (df)

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp37-none-any.whl size=11691 sha256=193cb704614fb0f4ade191c69b71307a32e9a14092f34d16fb4c2fe77968c868
  Stored in directory: C:\Users\haowenye\AppData\Local\pip\Cache\wheels\87\2a\18\4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
    Postcode           Borough          Neighbourhood
0        M1A      Not assigned           Not assigned
1        M2A      Not assigned           Not assigned
2        M3A        North York              Parkwoods
3        M4A        North York       Victoria Village
4        M5A  Downtown Toronto           Harbourfront
..       ...               ...                    ...
282      M8Z         Etobicoke              Mimico NW
283      M8Z         Etobicoke     The Queensway West
284      M8Z         Etobicoke  Royal York South West
285      M8Z         Etobicoke         South of Bloor
286      M9Z      Not assigned           Not assigned

[287 rows x 3 columns]


In [9]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


####  Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [22]:
data1=df[~(df['Borough']=='Not assigned')]

In [23]:
data1

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


######  More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [131]:
data2=data1.groupby('Postcode')

In [123]:
Borough=data2.Borough.unique()

In [145]:
data2=data1.groupby('Postcode').agg(','.join)

In [146]:
data2.Borough=Borough

In [147]:
data2

,Borough,Neighbourhood
Postcode,,
M1B,[Scarborough],"Rouge,Malvern"
M1C,[Scarborough],"Highland Creek,Rouge Hill,Port Union"
M1E,[Scarborough],"Guildwood,Morningside,West Hill"
M1G,[Scarborough],Woburn
M1H,[Scarborough],Cedarbrae
...,...,...
M9N,[York],Weston
M9P,[Etobicoke],Westmount
M9R,[Etobicoke],"Kingsview Village,Martin Grove Gardens,Richvie..."


In [148]:
## reset index
data2.reset_index(inplace=True)

In [149]:
data2

,Postcode,Borough,Neighbourhood
0,M1B,[Scarborough],"Rouge,Malvern"
1,M1C,[Scarborough],"Highland Creek,Rouge Hill,Port Union"
2,M1E,[Scarborough],"Guildwood,Morningside,West Hill"
3,M1G,[Scarborough],Woburn
4,M1H,[Scarborough],Cedarbrae
...,...,...,...
98,M9N,[York],Weston
99,M9P,[Etobicoke],Westmount
100,M9R,[Etobicoke],"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,[Etobicoke],"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


###### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [150]:
data2.loc[data2[data2.Neighbourhood=='Not assigned'].index,'Neighbourhood']=data2[data2.Neighbourhood=='Not assigned']['Borough']

In [151]:
## exame if there is Not assigned Neighbourhood
data2[data2.Neighbourhood=='Not assigned']

,Postcode,Borough,Neighbourhood


####  In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [152]:
data2.shape

(103, 3)

In [60]:
data2.shape[0]

103

#### load location file

In [104]:
new=pd.read_csv('http://cocl.us/Geospatial_data')

In [105]:
## change column names to merge data
new.columns=['Postcode','Latitude','Longitude']

In [157]:
Toronto=data2.merge(new)

In [158]:
Toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,[Scarborough],"Rouge,Malvern",43.806686,-79.194353
1,M1C,[Scarborough],"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,[Scarborough],"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,[Scarborough],Woburn,43.770992,-79.216917
4,M1H,[Scarborough],Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,[York],Weston,43.706876,-79.518188
99,M9P,[Etobicoke],Westmount,43.696319,-79.532242
100,M9R,[Etobicoke],"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,[Etobicoke],"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


In [173]:
for i in range(Toronto.shape[0]):
    Toronto.loc[i,'Borough']=Toronto.loc[i,'Borough'][0]

In [174]:
Toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


In [180]:
Toronto=Toronto.astype({'Latitude':'object','Longitude':'object'})

In [181]:
Toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
...,...,...,...,...,...
98,M9N,York,Weston,43.7069,-79.5182
99,M9P,Etobicoke,Westmount,43.6963,-79.5322
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.6889,-79.5547
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.7394,-79.5884


### Thank you for reviewing my IBM Capstone project1 